# Fine-tunning DeepLabV3 from SIMCLR on Parihaka Dataset

## Imports

In [1]:
from common import get_data_module, get_trainer_pipeline
import torch
from minerva.models.nets.image.deeplabv3 import DeepLabV3

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
/home/vscode/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Variables

In [3]:
root_data_dir = "/workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/images"
root_annotation_dir = "/workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/annotations"
ckpt_file = "/workspaces/HIAAC-KR-Dev-Container/shared_data/notebooks_e_pesos/pesos/simclr.ckpt"


img_size = (1006, 590)          # Change this to the size of the images in the dataset
model_name = "simclr"       # Model name (just identifier)
dataset_name = "seam_ai"        # Dataset name (just identifier)
single_channel = False          # If True, the model will be trained with single channel images (instead of 3 channels)

log_dir = "./logs"              # Directory to save logs
batch_size = 2                  # Batch size    
seed = 42                       # Seed for reproducibility
num_epochs = 100                # Number of epochs to train
is_debug = True                 # If True, only 3 batch will be processed for 3 epochs
accelerator = "gpu"             # CPU or GPU
devices = 1                     # Num GPUs

## Data Module (do not change)

In [4]:
data_module = get_data_module(
    root_data_dir=root_data_dir,
    root_annotation_dir=root_annotation_dir,
    img_size=img_size,
    batch_size=batch_size,
    seed=seed,
    single_channel=single_channel
)

data_module

DataModule
    Data: /workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/images
    Annotations: /workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/annotations
    Batch size: 2

In [5]:
# Just to check if the data module is working
data_module.setup("fit")
train_batch_x, train_batch_y = next(iter(data_module.train_dataloader()))
train_batch_x.shape, train_batch_y.shape

(torch.Size([2, 3, 1006, 590]), torch.Size([2, 1, 1006, 590]))

## **** Create and Load model HERE ****

In [6]:
model = DeepLabV3()
model

DeepLabV3(
  (backbone): DeepLabV3Backbone(
    (RN50model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu)

In [7]:
list(model.state_dict().keys())

['backbone.RN50model.conv1.weight',
 'backbone.RN50model.bn1.weight',
 'backbone.RN50model.bn1.bias',
 'backbone.RN50model.bn1.running_mean',
 'backbone.RN50model.bn1.running_var',
 'backbone.RN50model.bn1.num_batches_tracked',
 'backbone.RN50model.layer1.0.conv1.weight',
 'backbone.RN50model.layer1.0.bn1.weight',
 'backbone.RN50model.layer1.0.bn1.bias',
 'backbone.RN50model.layer1.0.bn1.running_mean',
 'backbone.RN50model.layer1.0.bn1.running_var',
 'backbone.RN50model.layer1.0.bn1.num_batches_tracked',
 'backbone.RN50model.layer1.0.conv2.weight',
 'backbone.RN50model.layer1.0.bn2.weight',
 'backbone.RN50model.layer1.0.bn2.bias',
 'backbone.RN50model.layer1.0.bn2.running_mean',
 'backbone.RN50model.layer1.0.bn2.running_var',
 'backbone.RN50model.layer1.0.bn2.num_batches_tracked',
 'backbone.RN50model.layer1.0.conv3.weight',
 'backbone.RN50model.layer1.0.bn3.weight',
 'backbone.RN50model.layer1.0.bn3.bias',
 'backbone.RN50model.layer1.0.bn3.running_mean',
 'backbone.RN50model.layer1.0.

In [8]:
ckpt = torch.load(ckpt_file, map_location="cpu")
list(ckpt["state_dict"].keys())

/tmp/ipykernel_717459/587373324.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_file, map_location="cpu")


['backbone.RN50model.conv1.weight',
 'backbone.RN50model.bn1.weight',
 'backbone.RN50model.bn1.bias',
 'backbone.RN50model.bn1.running_mean',
 'backbone.RN50model.bn1.running_var',
 'backbone.RN50model.bn1.num_batches_tracked',
 'backbone.RN50model.layer1.0.conv1.weight',
 'backbone.RN50model.layer1.0.bn1.weight',
 'backbone.RN50model.layer1.0.bn1.bias',
 'backbone.RN50model.layer1.0.bn1.running_mean',
 'backbone.RN50model.layer1.0.bn1.running_var',
 'backbone.RN50model.layer1.0.bn1.num_batches_tracked',
 'backbone.RN50model.layer1.0.conv2.weight',
 'backbone.RN50model.layer1.0.bn2.weight',
 'backbone.RN50model.layer1.0.bn2.bias',
 'backbone.RN50model.layer1.0.bn2.running_mean',
 'backbone.RN50model.layer1.0.bn2.running_var',
 'backbone.RN50model.layer1.0.bn2.num_batches_tracked',
 'backbone.RN50model.layer1.0.conv3.weight',
 'backbone.RN50model.layer1.0.bn3.weight',
 'backbone.RN50model.layer1.0.bn3.bias',
 'backbone.RN50model.layer1.0.bn3.running_mean',
 'backbone.RN50model.layer1.0.

In [9]:
from minerva.models.loaders import FromPretrained

model = FromPretrained(
    model,
    ckpt_path=ckpt_file,
    strict=False,
    filter_keys=["^backbone.RN50model."],
    error_on_missing_keys=False
)


Model loaded from /workspaces/HIAAC-KR-Dev-Container/shared_data/notebooks_e_pesos/pesos/simclr.ckpt
When loading model, the following keys are missing: ['fc.0.convs.0.0.weight', 'fc.0.convs.0.1.weight', 'fc.0.convs.0.1.bias', 'fc.0.convs.0.1.running_mean', 'fc.0.convs.0.1.running_var', 'fc.0.convs.1.0.weight', 'fc.0.convs.1.1.weight', 'fc.0.convs.1.1.bias', 'fc.0.convs.1.1.running_mean', 'fc.0.convs.1.1.running_var', 'fc.0.convs.2.0.weight', 'fc.0.convs.2.1.weight', 'fc.0.convs.2.1.bias', 'fc.0.convs.2.1.running_mean', 'fc.0.convs.2.1.running_var', 'fc.0.convs.3.0.weight', 'fc.0.convs.3.1.weight', 'fc.0.convs.3.1.bias', 'fc.0.convs.3.1.running_mean', 'fc.0.convs.3.1.running_var', 'fc.0.convs.4.1.weight', 'fc.0.convs.4.2.weight', 'fc.0.convs.4.2.bias', 'fc.0.convs.4.2.running_mean', 'fc.0.convs.4.2.running_var', 'fc.0.project.0.weight', 'fc.0.project.1.weight', 'fc.0.project.1.bias', 'fc.0.project.1.running_mean', 'fc.0.project.1.running_var', 'fc.1.weight', 'fc.2.weight', 'fc.2.bias',

## Pipeline

In [10]:
pipeline = get_trainer_pipeline(
    model=model,
    model_name=model_name,
    dataset_name=dataset_name,
    log_dir=log_dir,
    num_epochs=num_epochs,
    accelerator=accelerator,
    devices=devices,
    is_debug=is_debug,
    seed=seed,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 42


Log directory set to: /workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/simclr/seam_ai


In [11]:
pipeline.run(data_module, task="fit")

/usr/local/lib/python3.10/dist-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory ./logs/simclr/seam_ai exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/simclr/seam_ai/run_2024-12-19-18-37-46a5460df9e70b49bb8d2bbd640c62a96c.yaml



  | Name     | Type                    | Params | Mode 
-------------------------------------------------------------
0 | backbone | DeepLabV3Backbone       | 25.6 M | train
1 | fc       | DeepLabV3PredictionHead | 16.1 M | train
2 | loss_fn  | CrossEntropyLoss        | 0      | train
-------------------------------------------------------------
41.7 M    Trainable params
0         Non-trainable params
41.7 M    Total params
166.736   Total estimated model params size (MB)
186       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 3/3 [00:01<00:00,  2.70it/s, v_num=m_ai, val_loss=7.540, train_loss=1.020]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 3/3 [00:14<00:00,  0.21it/s, v_num=m_ai, val_loss=7.540, train_loss=1.020]
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/simclr/seam_ai/run_2024-12-19-18-37-46a5460df9e70b49bb8d2bbd640c62a96c.yaml


In [13]:
print(f"Checkpoint saved at {pipeline.trainer.checkpoint_callback.last_model_path}")

Checkpoint saved at ./logs/simclr/seam_ai/checkpoints/last.ckpt
